### Linear Regression

Other Sources:
- Here is a <a href="https://www.hackerearth.com/practice/machine-learning/machine-learning-algorithms/beginners-guide-regression-analysis-plot-interpretations/tutorial/">nice introductory article</a>.  It gives some additional attention to examining and evaluating the regression model. The code is written in R rather than Python, but it shouldn't be hard to interpret the few lines of R code. 
- Text section 18.6 cover the material theoretically and quickly.  Don't worry about the material on gradient descent.
- Chapter 3 of <a href="https://www-bcf.usc.edu/~gareth/ISL/ISLR%20First%20Printing.pdf">An Introduction to Statistical Learning</a> is your best bet for a deeper treatment, even though R is the language of choice.  In fact, that should be your go-to book for more machine learning material.

Objectives:
- Define data modeling and simple linear regression
- Show various steps in examining a data set and preparing it for input to a learning library
- Build a linear regression model using Python libraries
- Understand how to evaluate the quality of a model and compare it alternative models

##### The Bikeshare Data Set

We'll be working with a data set from Capital Bikeshare that was used in a [Kaggle competition](https://www.kaggle.com/c/bike-sharing-demand/data), but heavily modified for this class!

The goal is to predict total ridership of Capital Bikeshare in any given hour.

### Capital Bikeshare Data Dictionary

| Variable| Description |
|---------|----------------|
|rental_hour| hourly date + timestamp  |
|season|  'winter', 'spring', 'summer', 'fall' |
|workingday| whether the day is neither a weekend nor holiday|
|weather| 1 -> Clear or partly cloudy;  2 -> Clouds and mist; 3 -> Light rain or snow;  4 -> Heavy rain or snow|
|temp| temperature in Celsius|
|humidity| relative humidity|
|windspeed| wind speed|
|total_rentals| number of total rentals|
|sunspots| sunspot activity (0-2, none to high)|


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 14

In [2]:
bikes = pd.read_csv('bikeshare.csv', parse_dates=['rental_hour'])

In [4]:
bikes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 9 columns):
rental_hour      10886 non-null datetime64[ns]
season           10886 non-null object
workingday       10886 non-null int64
weather          10886 non-null int64
temp             10886 non-null float64
humidity         10886 non-null int64
windspeed        10886 non-null float64
total_rentals    10886 non-null int64
sunspots         10886 non-null int64
dtypes: datetime64[ns](1), float64(2), int64(5), object(1)
memory usage: 765.5+ KB


In [7]:
bikes.head(5)

,rental_hour,season,workingday,weather,temp,humidity,windspeed,total_rentals,sunspots
0,2011-01-01 00:00:00,winter,0,1,9.84,81,0.0,16,0
1,2011-01-01 01:00:00,winter,0,1,9.02,80,0.0,40,1
2,2011-01-01 02:00:00,winter,0,1,9.02,80,0.0,32,1
3,2011-01-01 03:00:00,winter,0,1,9.84,75,0.0,13,0
4,2011-01-01 04:00:00,winter,0,1,9.84,75,0.0,1,1


In [8]:
bikes.shape

(10886, 9)

In [3]:
bikes.describe()

,workingday,weather,temp,humidity,windspeed,total_rentals,sunspots
count,10886.000000,10886.000000,10886.00000,10886.000000,10886.000000,10886.000000,10886.000000
mean,0.680875,1.418427,20.23086,61.886460,12.799395,191.574132,0.998347
std,0.466159,0.633839,7.79159,19.245033,8.164537,181.144454,0.816232
min,0.000000,1.000000,0.82000,0.000000,0.000000,1.000000,0.000000
25%,0.000000,1.000000,13.94000,47.000000,7.001500,42.000000,0.000000
50%,1.000000,1.000000,20.50000,62.000000,12.998000,145.000000,1.000000
75%,1.000000,2.000000,26.24000,77.000000,16.997900,284.000000,2.000000
max,1.000000,4.000000,41.00000,100.000000,56.996900,977.000000,2.000000


In [6]:
bikes.season.value_counts()

fall      2734
summer    2733
spring    2733
winter    2686
Name: season, dtype: int64

### Linear Regression Basics
---

### Regression (versus Classification)

Both regression and classification are instances of *supervised learning*
  * We have "labeled training data"
  * Labeled data means we have an observations of X and observations of y
  * X are the "input values" and y is the "output value" or the "label"
  
Regression:  y is numeric

  * it makes sense to talk about $y_1 \geq y_2$ (ordinal) or $y \times 2$ (cardinal)
  * commonly ask "how much will $y$ *change* if $X$ changes?"
  
Classification:  y is *categorical*
  *  true or false
  *  red, yellow, blue, green, blue purple
  *  doesn't make sense to say "yellow plus one" or "halfway between blue and green"
  *  commonly ask "which value will $y$ be for this value of $X$?"
  
**The fact that a variable is coded as a number in your data frame does not make it numeric!**
  * code false/true as 0/1
  * code red, yellow, orange as 0/1/2
  * 0.5 is meaningless in both cases!
  

---

### Linear Regression

Think of our data frame as a set of observations of the form:  
$$(x_{i1}, x_{i2}, \ldots, x_{ik}, y_i)$$ 

where the $x$ are the "independent variables" and $y$ is the "dependent variable".  We are trying to find a function of the $x$ variables that predict the value of the $y$ variable.   We have $n$ observations and $k$ independent variables (features).

Think matrix with $n$ rows and $k$ columns for the independent variables and a vector with $n$ rows for the dependent variable.

*  *Regression*  the $y$ variable is numeric and ordered -- for example temperature or number of rentals
*  *Classification* the $y$ variable is taken from a set -- for example true/false or {red, green, blue}

*Linear* -- Start with the formula for a line:  $$y = \alpha + \beta x\\y = \beta_0 + \beta_1 x$$ 

(The second notation gives a consistent name to the coefficients.)

* First statement of the linear regression problem:  find the values of $\beta_0$ and $\beta_1$ that best predict the $y$ values for all of our observations
* Second statement of the problem:  since every model contains some amount of noise -- "random irreducible error" $\epsilon$,  instead we are actually working optimizing $$y = \beta_0 + \beta_1  + \epsilon$$
where $\epsilon$ is small and uncorrelated with $x$ and $y$.

---

Here, we will generalize this to $n$ independent variables as follows:

$$y = \beta_0 + \beta_1x_1 + \beta_2x_2 + ... + \beta_nx_n + \epsilon\quad =\quad \beta_0 + (\sum_{i=0}^{k} \beta_ix_i) + \epsilon$$

- $y$ is the response.
- $\beta_0$ is the intercept.
- $\beta_1$ is the coefficient for $x_1$ (the first feature).
- $\beta_k$ is the coefficient for $x_k$ (the kth feature).
- $\epsilon$ is the _error_ term which is independent of x, y, and $k$


A practical example of this applied to our data might be:

$${\tt total\_rides} = 20 + -2 \cdot {\tt temp} + -3 \cdot {\tt windspeed}\ +\ ...\ +\ 0.1 \cdot {\tt sunspots}$$

This equation is still called **linear** because the highest degree of the independent variables (e.g. $x_i$) is 1. Note that because the $\beta$ values are constants, they will not be independent variables in the final model, as seen above.  The $\beta_i$ are the parameters learned by the model.

#### Linearity Assumption

* Linear regression is very widely used, because it is fast and simple and well understood
* Even though most systems we model are not exactly linear
* But ultimately that's not what matters -- what matters is whether the model we build does a *good enough* job of predicting $y$
  * Means we can make good decisions about what to do based on our predictions
  
**Common Violations of Linearity**

* $y$ is a function of $x^2$, not $x$.  For example, rentals increase with ${\tt temp}^2$
* Interaction effects:  the effect of heat and humidity jointly affect rentals
* Effect depends on level:   for a while increase in **temp** causes **rentals** to increase, but if **temp** is high enough, **rentals** will start to decrease as **temp** increases

---

---

In the regression equation
$y = \beta_0 + \beta_1x_1 + \beta_2x_2 + ... + \beta_nx_k + \epsilon$

the $x$ and $y$ values come from data, and the $k+1$ $\beta$ values are called the **model coefficients**:

- The model coefficients are estimated (or "learned") during the model fitting process using the **least squares criterion**.
- Specifically, we are trying to find the line (mathematically) that minimizes the **sum of squared residuals** (or "sum of squared errors").
- Once we've learned these coefficients, we can use the model to predict the response for a new vector of independent variables:  $(x_1, x_2, \ldots, x_k)$

![Estimating coefficients](estimating_coefficients.png)

We measure how well the model fits the data in terms of mean-squared error as follows:
$$MSE = \frac{1} {n} \| \hat{y}(\mathbf{X}) - \vec{y} \|$$

The linear regression algorithm finds the values for $\beta_0, \beta_1,  \beta_2, \ldots \beta_n$ that maximize MSE *for a given data set (the training set)*

In the diagram above:

- The black dots are the **observed values** of x and y.
- The blue line is our **least squares line**.
- The red lines are the **residuals**, which are the vertical distances between the observed values and the least squares line.

In two dimensions (i.e. a single $x$ and a single $y$, i.e. we are learning $\beta_0$ and $\beta_1$) think of the learning process as
*  Moving the line up or down (changing the intercept $\beta_0$)
*  Changing the slope (increasing or decreasing $\beta_1$)

If there are two predictors $(x_1, x_2)$ then there are three parameters to learn, and the model fits a plane, and residuals are the distance between the observed value and the least squares plane.

![Regression with Two Variates](multiple_regression_plane.png)

If there are more than two predictors, it's hard to visualize.

#### On To Implementation

* $n$ observations (rows)
* Each observation has $m$ columns (one for each $x$ attribute) and an observed value for $y$
* $X$ is an $n\times m$ matrix,  $y$ is an $n\times 1$ vector


<a id="building-a-linear-regression-model-in-sklearn"></a>
### Building a (Single) Linear Regression Model in sklearn

#### Create a feature matrix called X that holds a `DataFrame` with only the temp variable and a `Series` called y that has the "total_rentals" column.

In [9]:
# Notice that we read both X and y in from the same file into the same data frame
#   But from the perspective of the learning algorithm they are completely different
#     so we immediately split them apart

feature_cols = ['temp']
X1 = bikes[feature_cols]
y1 = bikes.total_rentals

<span style="color:blue; font-size:120%">
Why do you think X is capitalized but y is not?

Important terminology.  Now (X, y) is our *training set* or our *training data*.

</span>

In [10]:
print(type(X1))
print(type(y1))
print(X1.shape)
print(y1.shape)

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>
(10886, 1)
(10886,)



### Scikit-learn's Four-Step Modeling Pattern -- Same for Many Learning Algorithms

**Step 1:** Import the class you plan to use.

In [12]:
from sklearn.linear_model import LinearRegression

**Step 2:** Create an instance of the algorithm 

In [13]:
lr1 = LinearRegression()
type(lr1)

sklearn.linear_model.base.LinearRegression

- Created an object that "knows" how to do linear regression, and is just waiting for data. This is an "unfitted" or "untrained" model

**Step 3:** Fit the model with data (aka "model training").

- Model is "learning" the relationship between X and y in our "training data."
- Process through which learning occurs varies by model.
- Occurs in-place.

In [14]:
lr1.fit(X1, y1)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

- Once a model has been fit with data, it's called a "fitted model."
- Next steps
  - We can predict y values from an X vector
  - We need to assess how good the model is as a predictor

In [15]:
print(lr1.intercept_)
print(lr1.coef_)

6.04621295961681
[9.17054048]


This is the equation

${\tt total\_rentals} = 6.04 + 9.17 \times{\tt temp}$

The intercept is $\beta_0$. The coefficient(s) $\beta_1$ is $9.17$.

The coefficient vector has length one only because we are demonstrating single linear regression.

In [16]:
# This is how we get our model to predict:  give it a matrix of size (n,m)
# In our case, m = 1 and we want to predict 3 values, so we need a 3x1 vector

values_to_predict = [0, 1, 2]
x_to_predict = np.array(values_to_predict).reshape(-1,1)
x_to_predict.shape

(3, 1)

In [17]:
# We can predict at multiple points.
# We should see the intercept at x=0 and a bump of 9.17 for every increase in X

print(lr1.predict(x_to_predict))

[ 6.04621296 15.21675344 24.38729392]


In [18]:
# Most often we want to predict all values in a matrix
print(lr1.predict(X1))

[ 96.2843313   88.7644881   88.7644881  ... 133.88354727 133.88354727
 126.36370408]


In [ ]:
# Those are our predicted values .... we will be interested in how they
#  compare to the actual (observed) y values
print(list(y1))

In [19]:
from sklearn.metrics import mean_squared_error
print(mean_squared_error(y1, lr1.predict(X1)))

27705.2238053288


Interpreting the intercept ($\beta_0$):

- It is the value of $y$ when all independent variables are 0.
- Here, it is the estimated number of rentals when the temperature is 0 degrees Celsius.
- <span style="color:blue">**Note:** It does not always make sense to interpret the intercept. (Why?)</span>

Interpreting the "temp" coefficient ($\beta_1$):

- **Interpretation:** An increase of 1 degree Celcius is _associated with_ increasing the number of total rentals by $\beta_1$.
- Here, a temperature increase of 1 degree Celsius is _associated with_ a rental increase of 9.17 bikes.
- This is not a statement of causation.
- $\beta_1$ would be **negative** if an increase in temperature was associated with a **decrease** in total rentals.
- $\beta_1$ would be **zero** if temperature is not associated with total rentals
  - Seeing a $\beta = 0$ is important to look for, it means that $x$ is not a useful variable and should be deleted!

#### Add More Features, Get Better Predictions!

In [20]:
bikes.columns

Index(['rental_hour', 'season', 'workingday', 'weather', 'temp', 'humidity',
       'windspeed', 'total_rentals', 'sunspots'],
      dtype='object')

rental_hour -> timestamp
season -> string
workingday -> 0/1
weather -> 1,2,3,4
sunspots -> 0,1,2

In [21]:
# These are all the numeric features -- regression needs numbers!
#  Omitting rental_hour (a date/time), omitting season (a string)
feature_cols2 = ['workingday', 'weather', 'temp', 'humidity', 'windspeed', 'sunspots']
X2 = bikes[feature_cols2]
y2 = bikes.total_rentals

In [22]:
lr2 = LinearRegression()
lr2.fit(X2, y2)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [23]:
print(lr2.intercept_)
print(lr2.coef_)

180.96434370104075
[-1.35048855  3.09471515  8.7489077  -2.75668603  0.32618071 -3.43731298]


In [ ]:
# Adding more variables makes a big difference ... temp is the 4th variable

In [24]:
print(lr1.intercept_)
print(lr1.coef_)

6.04621295961681
[9.17054048]


In [25]:
print(X1.shape)
print(X2.shape)

(10886, 1)
(10886, 6)


#####  Evaluating the Model -- First Cut

* Remember the regression algorithm is supposed to minimize MSE.
* Observe that MSE is a function only of the predicted and actual $y$ values
* Observe that scikit-learn makes it easy to compute these metrics
* Observe that adding the more variables makes MSE decrease.  Good!

In [26]:
from sklearn.metrics import mean_squared_error
print(mean_squared_error(y1, lr1.predict(X1)))
print(mean_squared_error(y2, lr2.predict(X2)))

27705.2238053288
24881.290399662896


####  A Quick Diversion:  MSE vs R-Squared

We are minimizing MSE, but MSE is sensitive to magnitudes of the X variables.
If we were to change our X values, it can change the MSE

  * Temperature in F instead of C
  * Multiplying 'sunspot activity' by 10
  
This intuitively doesn't affect the quality of the model.

Instead of MSE we use $R^2$ (R-squared) which normalizes for X magnitude
  * $R^2$ guaranteed to be $\leq$ 1
  * Values close to 1 are better
  * Value of 0 means model is no better than ignoring $X$ completely and predicting a $y$ value equal to the mean
  * Negative value means the model is *worse* than predicting at the mean


In [27]:
#  Proof that R^2 is 0 if we predict at the mean!
#  Notice that model evaluation works the same for any metric.

from sklearn.metrics import r2_score
mean = y2.mean()
ynull = np.full((len(y2),1), mean)
print(r2_score(y2, ynull))

0.0


In [28]:
#  Compare our two models
from sklearn.metrics import r2_score
print(r2_score(y1, lr1.predict(X1)))
print(r2_score(y2, lr2.predict(X2)))

0.15559367802794855
0.2416621840009524


#### Standard Regression Output

The **statsmodel** package gives more information about the regression.

In [29]:
import statsmodels.api as sm
# We can fit a line with or without an intercept.  Default for scikit-learn is to 
# fit an intercept.  Default for statsmodel is not to fit an intercept.

X2C = sm.add_constant(X2)
results = sm.OLS(y2, X2C).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:          total_rentals   R-squared:                       0.242
Model:                            OLS   Adj. R-squared:                  0.241
Method:                 Least Squares   F-statistic:                     577.8
Date:                Mon, 25 Nov 2019   Prob (F-statistic):               0.00
Time:                        17:13:08   Log-Likelihood:                -70540.
No. Observations:               10886   AIC:                         1.411e+05
Df Residuals:                   10879   BIC:                         1.411e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        180.9643      8.471     21.363      0.0

#####  Things of interest in the report

* $R^2$ (same)
* Adjusted $R^2$
  * Adding new variables to a model *always* makes $R^2$ decrease
  * But adding a new variable isn't always a good thing (adding a random $X$ attribute doesn't improve the model)
  * So Adjusted $R^2$ incorporates a penalty for number of variables
  * So we can compare two models on the same training set, one has more variables than the other
* F Statistic
  * Tests the hypothesis that all coefficients are = 0
  * If they are, the model has no predictive power i.e. it's useless
     * If it happens, you did something seriously wrong
     * In this case there is 0 probability that this model is useless
* T Statistics on the intercept and slopes
  * Tests the hypothesis that the variable's coefficient is 0
     * If it is, that particular variable is not contributing to the regression
  * The p statistic is a the probability that the coefficient is 0
     * So the intercept and temp and humidity are probably useful
     * Working day and weather are probably not useful
     * Windspeed and sunspots might or not be useful
  * The confidence intervals tell you the same thing
     * If the confidence interval on a coefficient does not contain 0, then there's at least a 95% chance that the true coefficient is different from 0 (i.e useful for prediction purposes)
     
##### Summary

* So far we have used only numeric attributes
* Our $R^2$ is not impressive
* We could eliminate several variables from the model with hurting its "quality"
  * Quality defined by $R^2$


###  Revisit Our Attributes

* 'rental_hour' a timestamp  -- OUT
* 'season'  a string (four values)  -- OUT
* 'workingday' a 0-1 boolean -- IN
* 'weather' ranges from 1 to 4 -- IN
* 'temp' real-valued -- IN
* 'humidity' real-valued -- IN
* 'windspeed' real-valued -- IN
* 'sunspots'  0 to 2 from "none" to "high" -- IN

Remember 
  * Regression assigns a coefficient to each $x_i$
  * The coefficient $\beta_i$ means "if I increase (decrease) $x_i$ by 1 unit, I expect $y$ to increase (decrease) by $\beta_i$ units
  
For which of these variables does this make sense?

### Numeric vs Categorical Variables

*  Numeric $x$
  *  It make sense to think about $x+1$ or even $x \div 2$
    * This is a property of the data.  For example, wind speed
  *  It always must be the case the $x$ increasing has the same effect on $y$
    * This is a property of the linear model.  For example, if we see a unit increase in windspeed, we predict an increase of 3.26 number of rentals, all other variables being equal
  
* Categorical $x$
  *  Selected from a (usually) small set of values
    *  \{ red, white, blue \}
    * \{ true, false \} 
  * It doesn't make sense to say "halfway between red and white"
  * It doesn't make sense to say that the change from red to white will have the same effect as the change from white to blue
    
***NOTICE:  Just because your data frame has something coded as an integer doesn't make it numeric!!***
***NOTICE:  If a variable is categorical, it can't stay in the model as is!!***

#####  Is our Y variable categorical or is it numeric?  Must it be one or the other always??

Which of our X variables are numeric, and which are categorical?


In [ ]:
bikes.info()

#### Dealing With the Categorical Variables

* season -- string (four values)
* workingday -- 0/1
* weather (1-4)
* sunspots (0-2)

##### Work on season first;  make it a number, then decide what to do next

In [30]:
bikes['season'].value_counts()

fall      2734
summer    2733
spring    2733
winter    2686
Name: season, dtype: int64

In [31]:
bikes['season'] = bikes.season.replace({'fall': 0, 'summer': 1, 'spring': 2, 'winter': 3})

In [32]:
bikes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 9 columns):
rental_hour      10886 non-null datetime64[ns]
season           10886 non-null int64
workingday       10886 non-null int64
weather          10886 non-null int64
temp             10886 non-null float64
humidity         10886 non-null int64
windspeed        10886 non-null float64
total_rentals    10886 non-null int64
sunspots         10886 non-null int64
dtypes: datetime64[ns](1), float64(2), int64(6)
memory usage: 765.5 KB


In [33]:
bikes.season.value_counts()

0    2734
2    2733
1    2733
3    2686
Name: season, dtype: int64

##### Dummy Variables

The problem is we don't want to talk about the effect of fall to summer the same as the effect of summer to summer to spring, and spring to winter in terms of their effects on number of rentals.  Instead we would like to talk about the "summer effect" separate from the "winter effect" etc.

* I have a categorical variable that takes on $n$ values
* Since it's categorical, the difference between 3 and 4 is not the same as the difference between 1 and 2
* All I want to say is that it has to take on exactly one of the $n$ values
* So for season we create 4 new variables, each of which is either 0 or 1


In [34]:
season_dummies = pd.get_dummies(bikes.season, prefix='season')

In [35]:
season_dummies.sample(n=5)

,season_0,season_1,season_2,season_3
7594,0,0,1,0
5741,0,0,0,1
5958,0,0,0,1
7521,0,0,1,0
5870,0,0,0,1


In [37]:
# And we replace the categorical variable in the data frame with the dummies.
#  BE SURE TO DROP THE ORIGINAL VARIABLE!!!!

bikes = pd.concat([bikes, season_dummies], axis=1)
bikes.drop(['season'], axis=1, inplace=True)

In [38]:
bikes.head(5)

,rental_hour,workingday,weather,temp,humidity,windspeed,total_rentals,sunspots,season_0,season_1,season_2,season_3
0,2011-01-01 00:00:00,0,1,9.84,81,0.0,16,0,0,0,0,1
1,2011-01-01 01:00:00,0,1,9.02,80,0.0,40,1,0,0,0,1
2,2011-01-01 02:00:00,0,1,9.02,80,0.0,32,1,0,0,0,1
3,2011-01-01 03:00:00,0,1,9.84,75,0.0,13,0,0,0,0,1
4,2011-01-01 04:00:00,0,1,9.84,75,0.0,1,1,0,0,0,1


<span style="color:blue; font-size:120%">

* What about workingday?  (it's kind of categorical)
* What about weather (it's kind of categorical)
* What about sunspots (it's kind of categorical)

</span>

#### Dealing with rental_hour (the timestamp)

* Rental hour is the hour at which the $y$ variable was measured
* It is strictly increasing over time
  * Is that like an index or row number?   And if so do we keep it?
  * Another question:  is there simply a time effect (business is expanding over time, independent of other factors)
* There certainly should be time effects (i.e. attributes that depend only on time / rental_hour)
  * workingday is a time effect
  * others?
  
Our job is 
* Figure out what is interesting about the timestamp information
* Transform it into one or more *numeric* variables that can be added 

##### Transformations on the timestamp

* The concept of "regular working hour" must be significant!
* The number of hours since opening might be significant if there is a general trend toward rentals increasing separate from any other effect in the data set


In [39]:
bikes.rental_hour[0]

Timestamp('2011-01-01 00:00:00')

In [40]:
# Verify that the hour attribute is actually hour of the day
(bikes.rental_hour.map(lambda t: t.hour).min(), bikes.rental_hour.map(lambda t: t.hour).max())

(0, 23)

In [41]:
daytime = bikes.rental_hour.apply(lambda t: t.hour > 8 and t.hour < 18).replace({False: 0, True:1})

In [42]:
bikes['daytime'] = daytime

In [43]:
bikes['hours_since_open'] = range(0, bikes.shape[0])

In [44]:
bikes.drop(['rental_hour'], axis=1, inplace=True)

In [45]:
bikes.head(5)

,workingday,weather,temp,humidity,windspeed,total_rentals,sunspots,season_0,season_1,season_2,season_3,daytime,hours_since_open
0,0,1,9.84,81,0.0,16,0,0,0,0,1,0,0
1,0,1,9.02,80,0.0,40,1,0,0,0,1,0,1
2,0,1,9.02,80,0.0,32,1,0,0,0,1,0,2
3,0,1,9.84,75,0.0,13,0,0,0,0,1,0,3
4,0,1,9.84,75,0.0,1,1,0,0,0,1,0,4


###  New Variables!
* Dummies for the season
* Working day (boolean)
* Hours since opening

In [46]:
bikes.columns

Index(['workingday', 'weather', 'temp', 'humidity', 'windspeed',
       'total_rentals', 'sunspots', 'season_0', 'season_1', 'season_2',
       'season_3', 'daytime', 'hours_since_open'],
      dtype='object')

In [48]:
feature_cols3 = list(bikes.columns)
feature_cols3.remove('total_rentals')
X3 = bikes[feature_cols3]
y3 = bikes.total_rentals

In [49]:
feature_cols3

['workingday',
 'weather',
 'temp',
 'humidity',
 'windspeed',
 'sunspots',
 'season_0',
 'season_1',
 'season_2',
 'season_3',
 'daytime',
 'hours_since_open']

In [50]:
X3C = sm.add_constant(X3)
results = sm.OLS(y3, X3C).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:          total_rentals   R-squared:                       0.348
Model:                            OLS   Adj. R-squared:                  0.347
Method:                 Least Squares   F-statistic:                     526.9
Date:                Mon, 25 Nov 2019   Prob (F-statistic):               0.00
Time:                        17:29:02   Log-Likelihood:                -69720.
No. Observations:               10886   AIC:                         1.395e+05
Df Residuals:                   10874   BIC:                         1.396e+05
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const               35.2552      8.050  

#####  Observations
* Model got a little better!
* Workingday had no effect!
* Hours since open had a significant but small positive effect
* How do we interpret coefficients of the season dummies?
  * winter, spring, summer, fall



In [60]:
feature_cols4 = list(bikes.columns)
feature_cols4.remove('workingday')
feature_cols4.remove('windspeed')
feature_cols4.remove('sunspots')
feature_col4.remove('total_rentals')
#  I'm deliberately leaving one out!

NameError: name 'feature_col4' is not defined

In [62]:
feature_cols4.remove('total_rentals')

In [64]:
# get rid of workingday, 
X4 = bikes[feature_cols4]
y4 = bikes.total_rentals

In [65]:
X4C = sm.add_constant(X4)
results = sm.OLS(y4, X4C).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:          total_rentals   R-squared:                       0.347
Model:                            OLS   Adj. R-squared:                  0.347
Method:                 Least Squares   F-statistic:                     723.2
Date:                Mon, 25 Nov 2019   Prob (F-statistic):               0.00
Time:                        17:40:38   Log-Likelihood:                -69724.
No. Observations:               10886   AIC:                         1.395e+05
Df Residuals:                   10877   BIC:                         1.395e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const               37.3324      7.275  

### Evaluating Models / Comparing Models
---

We can make linear models now, but how do we select the "best" model to use for our applications? 

These are three different questions:
1.  How well does the model fit the observed data (training set)
2.  How well will the model predict future observations
3.  What is the best model for the business scenario (i.e. taking into account the reward for predicting correctly and the penalty for making an error).

Accuracy on the training set may differ from accuracy when the model goes into production for two reasons
1.  The training set was too small -- important features in the population didn't show up enough to be noticed
2.  The model found spurious signal in the training set "total sales is higher on days that begin with 'T'." (Overfitting)


#### Training Accuracy vs Test Accuracy

* We have been measuring *training accuracy* -- we have been evaluating the model on the training data
* We are interested in *production accuracy* or *test accuracy* -- how well will the model work on a data set (X) it has not "seen before" -- i.e. that was not used for training
* Training accuracy might or might not be a good measure of test accuracy
  * How big is the training set
  * How representative is the training set of test or production data?

##### Train/Test Split

* Split the data set into a training set and a test set (usually 75%/25%, but that is a real trade-off)
* Use the training set only for training -- no evaluation
* Use the test set only for evaluation -- no training


In [59]:
X4.columns

Index(['weather', 'temp', 'humidity', 'total_rentals', 'season_0', 'season_1',
       'season_2', 'season_3', 'daytime', 'hours_since_open'],
      dtype='object')

In [66]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X4, y4, train_size=0.75, test_size=0.25)

In [55]:
X_train.shape

(8164, 10)

In [56]:
X_test.shape

(2722, 10)

In [67]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [68]:
print("Training error")
print(round(mean_squared_error(y_train, lr.predict(X_train)), 2))
print(round(r2_score(y_train, lr.predict(X_train)), 4))
print("")
print("Test error")
print(round(mean_squared_error(y_test, lr.predict(X_test)), 2))
print(round(r2_score(y_test, lr.predict(X_test)), 4))

Training error
21318.21
0.3473

Test error
21757.17
0.3457


#### Even better -- cross validation!

* A problem with train/test fit is that you lose some training data 
* Cross validation works like this (10-fold cross-validation)
* Divide the training set into 10 pieces (test/train split)
* For each piece
  * Train and test on this split
* Test accuracy is average accuracy over all the splits

In [69]:
from sklearn.model_selection import cross_val_score
lr = LinearRegression()
scores = cross_val_score(lr, X4, y4, cv=10, scoring='neg_mean_squared_error')

In [70]:
scores

array([ -7022.56929604, -11103.32750916, -19473.96496981, -15010.30709843,
       -11554.08967607, -14043.45542322, -32130.58123428, -37927.08423448,
       -43586.87350276, -27942.27827604])

In [71]:
print(round(-scores.mean(), 2), round(scores.std(), 2))

21979.45 11951.24


In [72]:
for folds in (2, 3, 10, 100, 1000, 5000, 10000):
    scores = cross_val_score(lr, X4, y4, cv=folds, scoring='neg_mean_squared_error')
    print(round(-scores.mean(), 2), round(scores.std(), 2))

39352.38 5227.66
22558.24 10071.94
21979.45 11951.24
21581.65 13048.27
21578.08 17785.66
22457.97 38127.32
22754.55 43228.4
